In [1]:
import psycopg2
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
with open('form5500_data/config.json') as f:
   conf = json.load(f)
   host = conf['host']
   database = conf['database']
   user = conf['user']
   passw = conf['passw']

In [3]:
conn_str = "host={} dbname={} user={} password={}".format(host, database, user, passw)

In [4]:
conn = psycopg2.connect(conn_str)

In [10]:
query = "SELECT sb.SB_TOT_PARTCP_CNT, sb.SB_PLAN_AT_RISK_IND, sb.SB_EFF_INT_RATE_PRCNT, sb.SB_PLAN_YEAR_BEGIN_DATE, sb.SB_FNDNG_SHORT_IND, \
       sb.SB_PR_YR_FNDNG_PRCNT, sb.SB_FNDNG_TGT_PRCNT, sb.SB_TOT_FNDNG_TGT_AMT, sb.SB_CURR_VALUE_AST_01_AMT, \
       sb.SB_PLAN_TYPE_CODE, f.BUSINESS_CODE \
FROM sb_full sb \
LEFT JOIN f5500_full f \
ON sb.SB_EIN = f.SPONS_DFE_EIN AND sb.SB_PN = f.SPONS_DFE_PN \
WHERE sb.SB_PLAN_YEAR_BEGIN_DATE BETWEEN '2017-01-01' AND '2017-12-31';"

In [11]:
eda_df = pd.read_sql(query, con=conn)

In [12]:
eda_df

,sb_tot_partcp_cnt,sb_plan_at_risk_ind,sb_eff_int_rate_prcnt,sb_plan_year_begin_date,sb_fndng_short_ind,sb_pr_yr_fndng_prcnt,sb_fndng_tgt_prcnt,sb_tot_fndng_tgt_amt,sb_curr_value_ast_01_amt,sb_plan_type_code,business_code
0,3,0,6.12,2017-01-01,2,150.68,159.40,879299.0,1401678.0,1,None
1,3,None,4.16,2017-01-01,2,106.90,106.88,552939.0,591006.0,1,None
2,3,None,5.72,2017-01-01,2,148.47,152.62,670184.0,1022867.0,1,None
3,5,None,5.34,2017-01-01,2,104.99,104.07,1634026.0,1700543.0,1,None
4,3,None,5.75,2017-03-01,1,84.43,85.24,1315484.0,1134430.0,1,None
5,5,None,5.14,2017-01-01,2,106.50,112.68,2778442.0,3130817.0,1,None
6,21,None,5.90,2017-01-01,2,139.62,169.94,1231748.0,2093267.0,1,None
7,4,None,6.48,2017-01-01,2,80.00,215.99,74935.0,161859.0,1,None
8,3,None,4.57,2017-01-01,2,80.00,96.39,156983.0,151324.0,1,None
9,7,None,6.05,2017-01-01,1,0.00,136.48,391207.0,533957.0,1,None


<h3>Add calculated FS (using MVA instead of AVA) column</h3>

In [ ]:
eda_df['mva_fs'] = eda_df['sb_curr_value_ast_01_amt'] / eda_df['sb_tot_fndng_tgt_amt'] *100

<h3>Add year column (year only of plan year begin date) </h3>

In [ ]:
eda_df['plan_year_dt'] = pd.to_datetime(eda_df['sb_plan_year_begin_date'])

In [ ]:
eda_df['year'] = eda_df['plan_year_dt'].dt.year

<h3>Create integer versions of feature columns</h3>

<h3>Participant count</h3>

In [ ]:
eda_df['ptp_cnt'] = pd.to_numeric(eda_df['sb_tot_partcp_cnt'])

<h3>Prior year shortfall indicator</h3>

In [ ]:
eda_df['sb_fndng_short_ind'].unique()

In [ ]:
eda_df['sf_ind'] = pd.to_numeric(eda_df['sb_fndng_short_ind'])
eda_df['sf_ind'] = eda_df['sf_ind'].fillna(value=0)

<h1>Create Features DF</h1>
<h3>Drop rows with NA</h3>

In [ ]:
prelim_feat_df = eda_df[['ptp_cnt', 'sb_eff_int_rate_prcnt', 'year','sf_ind', \
                         'sb_pr_yr_fndng_prcnt', 'sb_tot_fndng_tgt_amt','mva_fs', \
                          'sb_plan_type_code','sb_fndng_tgt_prcnt']]

In [ ]:
prelim_feat_df.info()

In [ ]:
prelim_feat_df

In [ ]:
prelim_feat_df = prelim_feat_df.dropna(axis=0)

In [ ]:
y_ava_fs = prelim_feat_df['sb_fndng_tgt_prcnt'].values
y_fs = prelim_feat_df['mva_fs'].values
y_ft = prelim_feat_df['sb_tot_fndng_tgt_amt'].values

In [ ]:
X = prelim_feat_df[['ptp_cnt', 'sb_eff_int_rate_prcnt', 'sf_ind', \
                         'sb_pr_yr_fndng_prcnt']]

In [ ]:
X.info()

<h1>Features EDA</h1>

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
X2 = prelim_feat_df[['ptp_cnt', 'sb_eff_int_rate_prcnt', 'sf_ind', \
                         'sb_pr_yr_fndng_prcnt', 'sb_fndng_tgt_prcnt', \
                          'sb_plan_type_code']]

In [ ]:
plt.style.use(['dark_background'])
scatter_matrix(X2, alpha=0.6, figsize=(12, 12), diagonal='hist', hist_kwds={'bins':100})
plt.show()

<h3>EIR - restrict to values between 1 and 10 (appear to be invalid data entries)</h3>

In [ ]:
X2[X2['ptp_cnt']<=100].describe()

In [ ]:
plt.hist(X2[(X2['ptp_cnt']<4000) & (X2['ptp_cnt']>100)]['ptp_cnt'],bins=100)
plt.title('Distribution of Participant Count (100 < PC < 4000)')
plt.show()

In [ ]:
eir_nonzero = X2[(X2['sb_eff_int_rate_prcnt'] >1) & (X2['sb_eff_int_rate_prcnt'] < 10) \
                & (X2['sb_fndng_tgt_prcnt'] > 0) & (X2['sb_fndng_tgt_prcnt'] < 400) & (X2['sb_plan_type_code'] == '1')]
y_eirnonzero = eir_nonzero['sb_fndng_tgt_prcnt'].values

In [ ]:
eir_nonzero.shape, y_eirnonzero.shape, eir_nonzero['sb_eff_int_rate_prcnt'].shape

In [ ]:
eir_nonzero['sb_eff_int_rate_prcnt'].describe()

In [ ]:
plt.hist(eir_nonzero['sb_eff_int_rate_prcnt'],bins=100)
plt.title('Distribution of EIR (1.0 < EIR < 10.0)')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
#plt.scatter(X['sb_pr_yr_fndng_prcnt'], y_fs)
plt.scatter(eir_nonzero['sb_eff_int_rate_prcnt'], y_eirnonzero)
plt.xlabel('Effective Interest Rate')
plt.ylabel('Funded Status (AVA basis)')
plt.title('FS vs EIR')
plt.show()

In [ ]:
pd.DataFrame(eir_nonzero['sb_eff_int_rate_prcnt']).boxplot(sym=)
plt.show()

In [ ]:
sns.boxplot(pd.DataFrame(eir_nonzero['sb_eff_int_rate_prcnt']))
plt.title('Boxplot of EIR')
plt.show()

In [ ]:
eir_nonzero['sb_eff_int_rate_prcnt'].describe()

In [ ]:
sns.boxplot(prelim_feat_df['sb_fndng_tgt_prcnt'])
plt.title('Boxplot of FS - all plans')
plt.show()

In [ ]:
sns.boxplot(eir_nonzero['sb_fndng_tgt_prcnt'])
plt.title('Boxplot of FS (0<FS<400%)')
plt.show()